In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [ ]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf


In [ ]:
%matplotlib inline

In [ ]:
# sec example dataset with correlations
df = pd.read_csv('comparisons.csv.gz')

In [ ]:
pf = df.query("norm_cossimw >=0.1 and norm_cossimn>=0.1")
data = pf[['norm_cossimw', 'norm_cossimn', 'norm_jaro', 'norm_ed']][pf.diff_category=='unknown'].values.copy()

In [ ]:
# ranges for pdf normalization
# none means set autimatically.
x_min = [0] * 4
x_max = [1.] * 4
x_min[0] = 0.1
x_min[1] = 0.1
x_max[3] = None


In [ ]:
kde = KDECopulaNNPdf(x_min=x_min, x_max=x_max, rho=0.3)


In [ ]:
kde = kde.fit(data)

In [ ]:
X_g = kde.pipe_[0].transform(data)
plt.hist(X_g[:, 1], bins=50)

In [ ]:
X_p = kde.pipe_[0:2].transform(data)
plt.hist(X_p[:, 0], bins=50)

In [ ]:
X_u = kde.pipe_.transform(data)
plt.hist(X_u[:, 1], bins=50)

In [ ]:
# after transformations: pca and uniform datasets
pf = pd.DataFrame(X_p)
df = pd.DataFrame(X_u)

In [ ]:
import phik

In [ ]:
pf.corr()

In [ ]:
pf.phik_matrix()

In [ ]:
df.corr()

In [ ]:
df.phik_matrix()

In [ ]:
df.significance_matrix()

In [ ]:
# use mutual information to capture residual levels of non-linearity
from sklearn.feature_selection import mutual_info_regression
mi = np.zeros((4, 4))
for i in range(4):
    mi[i, :] = mutual_info_regression(X_u, X_u[:, i])

mi

In [ ]:
X_gen, sample_weight = kde.sample(200000)

In [ ]:
sum(sample_weight)

In [ ]:
plt.hist(sample_weight, bins=50, log=True)

In [ ]:
i=1
plt.figure(figsize=(12, 7))
plt.hist(X_gen[:, i], bins=100, range=(0,1), density=True, alpha=0.5, weights=sample_weight)
plt.hist(data[:, i], bins=50, range=(0,1), density=True, alpha=0.5)

In [ ]:
i=3
plt.figure(figsize=(12, 7))
plt.hist(X_gen[:, i], bins=100, range=(0,100), density=True, alpha=0.5, weights=sample_weight)
plt.hist(data[:, i], bins=50, range=(0,100), density=True, alpha=0.5)

In [ ]:
kde.score(data)

In [ ]:
X_gen, sample_weight = kde.sample(12195)

In [ ]:
i=1
j=2
plt.figure(figsize=(12,7))
plt.scatter(X_gen[:,i], X_gen[:,j], s=0.2*sample_weight)

In [ ]:
plt.figure(figsize=(12,7))
plt.scatter(data[:,i], data[:,j], s=0.2)